In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter, AutoDateLocator, SecondLocator
from time_management import *
from data_management import *


def efficient_moving_average_multiAvg(times, data, window_size_seconds, sampling_rate):
    window_size_samples = max(1, int(window_size_seconds * sampling_rate))  # Ensure minimum window size of 1
    
    if window_size_samples == 1:
        # For extremely small windows, just return the original data
        return np.array(data)
    
    # Use rolling to apply the smoothing
    smoothed_data = pd.Series(data).rolling(
        window=window_size_samples, 
        center=True, 
        min_periods=1
    ).mean().to_numpy()
    
    # Handle edge effects
    half_window = window_size_samples // 2
    smoothed_data[:half_window] = smoothed_data[half_window]
    smoothed_data[-half_window:] = smoothed_data[-half_window-1]
    
    return smoothed_data

def set_magnetic_hole_plot_title(trange):
    """
    Generate the plot title for magnetic hole analysis based on the time range.

    Parameters:
    trange (list): A list containing two time strings in the format 'YYYY-MM-DD/HH:MM:SS' or 'YYYY-MM-DD/HH:MM:SS.fff'

    Returns:
    str: The formatted plot title
    """
    start_time, stop_time = trange
    
    # Convert the time range to formatted strings
    start_datetime_str, stop_datetime_str = convert_time_range_to_str(start_time, stop_time)
    
    # Determine the encounter number
    start_date = '-'.join(start_datetime_str.split('_')[0:3])
    encounter_number = get_encounter_number(start_date)

    start_date, start_time = start_time.split('/')
    stop_date, stop_time = stop_time.split('/')

    # Function to format time without trailing zeros for milliseconds
    def format_time(time_str):
        if '.' in time_str:
            time_parts = time_str.split('.')
            milliseconds = time_parts[1].rstrip('0')
            return f"{time_parts[0]}.{milliseconds}" if milliseconds else time_parts[0]
        return time_str

    start_time = format_time(start_time)
    stop_time = format_time(stop_time)

    if start_date == stop_date:
        plot_title = f"PSP Magnetic Hole Analysis {encounter_number} {start_date} {start_time} to {stop_time}"
    else:
        if start_date[:4] == stop_date[:4]:  # Same year
            plot_title = f"PSP Magnetic Hole Analysis {encounter_number} {start_date} {start_time} to {stop_date[5:]} {stop_time}"
        else:
            plot_title = f"PSP Magnetic Hole Analysis {encounter_number} {start_time} to {stop_time}"

    return plot_title

def generate_heatmap_for_trange(times_clipped, bmag, window_sizes, sampling_rate, threshold, trange, top_plot_smoothing_window):
    bool_heatmap = []
    float_heatmap = []

    # Calculate smoothed data for the top plot
    smoothed_data_for_top_plot = efficient_moving_average_multiAvg(
        times_clipped,
        bmag,
        top_plot_smoothing_window,
        sampling_rate
    )

    for window_size in window_sizes:
        smoothed_data = efficient_moving_average_multiAvg(
            times_clipped,
            bmag,
            window_size,
            sampling_rate
        )
        
        # Calculate the difference between bmag and smoothed data * threshold
        diff_data = bmag - smoothed_data * threshold
    
        # Convert to boolean values (1 when bmag < smoothed value * threshold, 0 otherwise)
        bool_diff = (diff_data < 0).astype(int)
    
        # Append the results to the heatmap arrays
        bool_heatmap.append(bool_diff)
        float_heatmap.append(diff_data)
    
    # Convert to numpy arrays for plotting
    bool_heatmap = np.array(bool_heatmap)
    float_heatmap = np.array(float_heatmap)
    
    print(f"Boolean heatmap shape: {bool_heatmap.shape}")
    print(f"Float heatmap shape: {float_heatmap.shape}")
    
    # Convert times_clipped to matplotlib date format
    times_num = mdates.date2num(times_clipped)

    # Plotting the line plot and heatmaps
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    # Set the main title using the new function
    fig.suptitle(set_magnetic_hole_plot_title(trange), fontsize=10, fontweight='bold')
    
    # Plot the raw and smoothed data at the top
    plot_raw_and_smoothed_data(times_num, bmag, smoothed_data_for_top_plot, axes[0], threshold, top_plot_smoothing_window)
    axes[0].set_xlim(times_num[0], times_num[-1])  # Align x-axis with the heatmaps


    # Plot the boolean heatmap
    im1 = axes[1].imshow(np.flipud(bool_heatmap), aspect='auto', cmap='binary', 
                   extent=[times_num[0], times_num[-1], 0, len(window_sizes)-1])
    axes[1].set_title(f'Boolean Heatmap (1 = bmag < smoothed * {threshold:.2f})')
    axes[1].set_ylabel('Smoothing Window Index')
    axes[1].axhline(y=10, color='black', linestyle='--', linewidth=0.8)
    
    # Add legend/key for the boolean heatmap
    hole_patch = mpatches.Patch(color='black', label=f'Hole (bmag < smoothed * {threshold:.2f})')
    non_hole_patch = mpatches.Patch(color='white', label=f'No Hole (bmag >= smoothed * {threshold:.2f})')
    axes[1].legend(handles=[hole_patch, non_hole_patch], loc='lower right')

    # Plot the float heatmap
    im2 = axes[2].imshow(np.flipud(float_heatmap), aspect='auto', cmap='seismic', 
                        extent=[times_num[0], times_num[-1], 0, len(window_sizes)-1])
    axes[2].set_title(f'Float Heatmap (bmag - smoothed * {threshold:.2f})')
    axes[2].set_ylabel('Smoothing Window Index')
    axes[2].set_xlabel('Time')
    # axes[2].axhline(y=1, color='grey', linestyle='--', linewidth=0.8)  # Add light horizontal line at y=1
    axes[2].axhline(y=10, color='black', linestyle='--', linewidth=0.8)

    # Add colorbars to the heatmaps
    # fig.colorbar(im1, ax=axes[1])
    # fig.colorbar(im2, ax=axes[2])

    # Set the date format for the x-axis
    time_format = DateFormatter('%H:%M:%S.')
    locator = SecondLocator()

    # Apply custom format to each axis
    for ax in axes:
        ax.xaxis.set_major_formatter(plt.FuncFormatter(format_date))
        ax.xaxis.set_major_locator(locator)

    # Set y-ticks to show actual window sizes
    y_ticks = np.arange(len(window_sizes))
    y_tick_labels = [f'{w:.2f}' for w in window_sizes]
    for ax in axes[1:]:
        ax.set_yticks(y_ticks)
        ax.set_yticklabels(y_tick_labels)
        ax.set_ylabel('Smoothing Window (seconds)')

    # Show the plot
    plt.tight_layout()
    plt.show()

# Define a custom format function to truncate microseconds to 3 digits
def format_date(x, pos=None):
    return mdates.num2date(x).strftime('%-H:%M:%S')  # Truncate to milliseconds and remove leading zero

# Function to plot the raw and smoothed data
def plot_raw_and_smoothed_data(times_clipped, bmag, smoothed_data, axes, threshold, TOP_PLOT_SMOOTHING_WINDOW):
    axes.plot(times_clipped, bmag, color='black', label='Raw Data')
    axes.plot(times_clipped, smoothed_data, color='red', label=f'{TOP_PLOT_SMOOTHING_WINDOW}s Smoothed Data')
    axes.plot(times_clipped, smoothed_data * threshold, color='green', linestyle='--', label=f'Threshold ({threshold:.2f})')
    axes.set_title(f' Raw and {TOP_PLOT_SMOOTHING_WINDOW}s Smoothed Data')
    axes.set_xlabel('Time')
    axes.set_ylabel('Bmag')
    axes.legend(loc='lower right')

# Main execution
if __name__ == "__main__":
    # Constants
    INSTRUMENT_SAMPLING_RATE = 297.2  # Hz, for millisecond precision
    threshold = 0.8  # Threshold parameter
    TOP_PLOT_SMOOTHING_WINDOW = 1  # or whatever value you want to use

    # Define window sizes on a logarithmic scale
    window_sizes = np.logspace(-1, 1, num=21)  # 20 points between 10^-1 and 10^1

    # Set time range
    # trange = ['2023-09-28/06:37:15.000', '2023-09-28/06:37:33.000']  # wide lots o dips÷

    # trange = ['2023-09-28/06:10:00.000', '2023-09-28/07:10:00.000'] #Whole Shebang
    # trange = ['2023-09-28/06:37:05.500', '2023-09-28/06:37:08.000'] #Interesting Test Case WE'VE BEEN USING for Derivative
    # trange = ['2023-09-28/06:39:00.000', '2023-09-28/06:40:10.000'] #⭐️⭐️⭐️ GREAT region for solid hole testing⭐️⭐️⭐️
    trange = ['2023-09-28/06:39:45.000', '2023-09-28/06:40:02.000'] #⭐️⭐️⭐️ GREAT region for solid hole testing⭐️⭐️⭐️
    # trange = ['2023-09-28/06:39:07.000', '2023-09-28/06:39:10.000'] #WONKY WONKS
    # trange = ['2023-09-28/06:39:00.000', '2023-09-28/06:39:20.000'] #WONKY WONKS bigger

    # trange = ['2023-09-28/06:39:50.000', '2023-09-28/06:39:55.000'] #⭐️⭐️⭐️ Checking a single hole
    # trange = ['2023-09-28/06:39:31.000', '2023-09-28/06:39:35.000'] #⭐️⭐️⭐️ PERFECT Single hole for testing⭐️⭐️⭐️
    # trange = ['2023-09-28/06:39:45.000', '2023-09-28/06:39:50.000'] #⭐️⭐️⭐️ ANOTHER PERFECT Single hole for testing⭐️⭐️⭐️
    # trange = ['2023-09-28/06:37:05.500', '2023-09-28/06:37:08.000'] #Interesting Test Case WE'VE BEEN USING for Derivative
    # trange = ['2023-09-28/06:36:022', '2023-09-28/06:36:24'] # A pretty hole ⭐️
    # trange = ['2023-09-28/06:29:18', '2023-09-28/06:29:22'] #with a larger window #🇲🇦 Asymmetry between peaks: 42.25%

    # Download and prepare the high-res Parker Solar Probe magnetic field data
    extended_trange = extend_time_range(trange, max(window_sizes))
    times, br, bt, bn, bmag_extended = download_and_prepare_high_res_mag_data(extended_trange)
    sampling_rate = determine_sampling_rate(times, INSTRUMENT_SAMPLING_RATE, True)

    # Clip the data based on the original trange
    times_clipped, bmag = clip_to_original_time_range(times, bmag_extended, trange)

    # Now generate the heatmap with the Parker Solar Probe data
    generate_heatmap_for_trange(times_clipped, bmag, window_sizes, sampling_rate, threshold, trange, TOP_PLOT_SMOOTHING_WINDOW)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates  # Import for date handling
from matplotlib.dates import DateFormatter, AutoDateLocator

import multiAvg_calc
from multiAvg_calc import *

import data_management
from data_management import *

# Function to generate the heatmap for the entire trange
def generate_heatmap_for_trange(times_clipped, bmag, base_window_seconds, step_size_seconds, num_steps, sampling_rate):
    bool_heatmap = []
    float_heatmap = []
    smoothed_data = None  # Initialize for use in the line plot

    for step in range(num_steps):
        current_window = base_window_seconds + step * step_size_seconds
        smoothed_data = efficient_moving_average_multiAvg(
            times_clipped,
            bmag,
            current_window,
            sampling_rate
        )
        
        # Calculate the difference between bmag and smoothed data
        diff_data = bmag - smoothed_data
    
        # Convert to boolean values (1 when bmag < smoothed value, 0 otherwise)
        bool_diff = (diff_data < 0).astype(int)
    
        # Append the results to the heatmap arrays
        bool_heatmap.append(bool_diff)
        float_heatmap.append(diff_data)
    
    # Convert to numpy arrays for plotting
    bool_heatmap = np.array(bool_heatmap)
    float_heatmap = np.array(float_heatmap)
    
    print(f"Boolean heatmap shape: {bool_heatmap.shape}")
    print(f"Float heatmap shape: {float_heatmap.shape}")
    
    # Convert times_clipped to matplotlib date format
    times_num = mdates.date2num(times_clipped)

    # Plotting the line plot and heatmaps
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))  # Add a 3rd plot for raw and smoothed data
    
    # Plot the raw and smoothed data at the top
    plot_raw_and_smoothed_data(times_num, bmag, smoothed_data, axes[0])
    axes[0].set_xlim(times_num[0], times_num[-1])  # Align x-axis with the heatmaps

    # Plot the boolean heatmap
    axes[1].imshow(np.flipud(bool_heatmap), aspect='auto', cmap='binary', 
                   extent=[times_num[0], times_num[-1], base_window_seconds, base_window_seconds + step_size_seconds * num_steps])
    axes[1].set_title('Boolean Heatmap (1 = bmag < smoothed)')
    axes[1].set_ylabel('Smoothing Window (seconds)')
    
    # Add legend/key for the boolean heatmap
    hole_patch = mpatches.Patch(color='black', label='Hole (bmag < smoothed)')
    non_hole_patch = mpatches.Patch(color='white', label='No Hole (bmag >= smoothed)')
    axes[1].legend(handles=[hole_patch, non_hole_patch], loc='upper right')

    # Plot the float heatmap
    im = axes[2].imshow(np.flipud(float_heatmap), aspect='auto', cmap='seismic', 
                        extent=[times_num[0], times_num[-1], base_window_seconds, base_window_seconds + step_size_seconds * num_steps])
    axes[2].set_title('Float Heatmap (bmag - smoothed)')
    axes[2].set_ylabel('Smoothing Window (seconds)')
    axes[2].set_xlabel('Time')

    # Set the date format for the x-axis
    time_format = DateFormatter('%H:%M:%S.%f')
    locator = AutoDateLocator()

    # Apply custom format to each axis
    for ax in axes:
        ax.xaxis.set_major_formatter(plt.FuncFormatter(format_date))
        ax.xaxis.set_major_locator(locator)

    # Show the plot
    plt.tight_layout()
    plt.show()

# -------- Smoothing Function for Multi-Average Heatmap -------- #
# Define a custom format function to truncate microseconds to 3 digits
def format_date(x, pos=None):
    return mdates.num2date(x).strftime('%H:%M:%S.%f')[:-3]  # Truncate to milliseconds

# Function to plot the raw and smoothed data
def plot_raw_and_smoothed_data(times_clipped, bmag, smoothed_data, axes):
    axes.plot(times_clipped, bmag, color='black', label='Raw Data')
    axes.plot(times_clipped, smoothed_data, color='red', label='Smoothed Data')
    axes.set_title('Raw and Smoothed Data')
    axes.set_xlabel('Time')
    axes.set_ylabel('Bmag')
    axes.legend(loc='upper right')

# -------- Clip Data to Original Time Range for Multi-Average Heatmap -------- #
def clip_to_original_time_range_multiHeatmap(times, data, trange):
    times_clipped, data_clipped = clip_to_original_time_range_multiAvg(times, data, trange)
    return times_clipped, data_clipped

# Constants
base_window_seconds = 0.1  # Base window size in seconds
step_size_seconds = 0.1  # Step size in seconds
num_steps = 50  # Number of steps
INSTRUMENT_SAMPLING_RATE = 1000  # 1000 Hz, for millisecond precision
# trange = ['2023-09-28 06:39:00.000', '2023-09-28 06:39:05.000']  # Adjusted trange to match millisecond precision
#--------⏱️ Set time range--------//
# Define the input parameters
# trange = ['2023-09-28/06:37:15.000', '2023-09-28/06:37:45.000'] #wide lots o dips
# trange = ['2023-09-28/06:37:05.500', '2023-09-28/06:37:08.000'] #Interesting Test Case WE'VE BEEN USING for Derivative
trange = ['2023-09-28/06:39:00.000', '2023-09-28/06:40:10.000'] #⭐️⭐️⭐️ GREAT region for solid hole testing⭐️⭐️⭐️
# trange = ['2023-09-28/06:39:07.000', '2023-09-28/06:39:10.000'] #WONKY WONKS
# trange = ['2023-09-28/06:39:50.000', '2023-09-28/06:39:55.000'] #⭐️⭐️⭐️ Checking a single hole
# trange = ['2023-09-28/06:39:31.000', '2023-09-28/06:39:35.000'] #⭐️⭐️⭐️ PERFECT Single hole for testing⭐️⭐️⭐️
# trange = ['2023-09-28/06:39:45.000', '2023-09-28/06:39:50.000'] #⭐️⭐️⭐️ ANOTHER PERFECT Single hole for testing⭐️⭐️⭐️
# trange = ['2023-09-28/06:37:05.500', '2023-09-28/06:37:08.000'] #Interesting Test Case WE'VE BEEN USING for Derivative

# trange = ['2023-09-28/06:36:022', '2023-09-28/06:36:24'] # A pretty hole ⭐️
# trange = ['2023-09-28/06:29:18', '2023-09-28/06:29:22'] #with a larger window #🇲🇦 Asymmetry between peaks: 42.25%

# Download and prepare the high-res Parker Solar Probe magnetic field data
extended_trange = extend_time_range_multiAvg(trange, max(base_window_seconds, step_size_seconds * num_steps))
times, br, bt, bn, bmag_extended = download_and_prepare_high_res_mag_data(extended_trange)
sampling_rate = determine_sampling_rate_multiAvg(times, INSTRUMENT_SAMPLING_RATE, True)

# Clip the data based on the original trange
times_clipped, bmag = clip_to_original_time_range_multiHeatmap(times, bmag_extended, trange)

# Now generate the heatmap with the Parker Solar Probe data
generate_heatmap_for_trange(times_clipped, bmag, base_window_seconds, step_size_seconds, num_steps, sampling_rate)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates  # Import for date handling
from matplotlib.dates import DateFormatter, AutoDateLocator

# -------- Smoothing Function for Multi-Average Heatmap -------- #
# Define a custom format function to truncate microseconds to 3 digits
def format_date(x, pos=None):
    return mdates.num2date(x).strftime('%H:%M:%S.%f')[:-3]  # Truncate to milliseconds


# Function to plot the raw and smoothed data
def plot_raw_and_smoothed_data(times_clipped, bmag, smoothed_data, axes):
    axes.plot(times_clipped, bmag, color='black', label='Raw Data')
    axes.plot(times_clipped, smoothed_data, color='red', label='Smoothed Data')
    axes.set_title('Raw and Smoothed Data')
    axes.set_xlabel('Time')
    axes.set_ylabel('Bmag')
    axes.legend(loc='upper right')

def efficient_moving_average_multiHeatmap(times, data, window_size_seconds, sampling_rate):
    window_size_samples = int(window_size_seconds * sampling_rate)  # Calculate window size in number of samples
    # Use rolling to apply the smoothing
    smoothed_data = pd.Series(data).rolling(window=window_size_samples, center=True, min_periods=1).mean().to_numpy()
    return smoothed_data

# -------- Clip Data to Original Time Range for Multi-Average Heatmap -------- #
def clip_to_original_time_range_multiHeatmap(times, data, trange):
    # Convert times to pandas DatetimeIndex if not already
    if not isinstance(times, pd.DatetimeIndex):
        times = pd.to_datetime(times)
    
    # Convert the trange to pandas Timestamps
    start_time = pd.to_datetime(trange[0])
    end_time = pd.to_datetime(trange[1])
    
    # Create a mask for clipping
    mask = (times >= start_time) & (times <= end_time)
    times_clipped = times[mask]
    data_clipped = data[mask]
    
    return times_clipped, data_clipped

# Constants
base_window_seconds = .1  # Base window size in seconds
step_size_seconds = 0.01  # Step size in seconds
num_steps = 100  # Number of steps
sampling_rate = 1000  # 1000 Hz, for millisecond precision
n_points = 5000  # Number of data points
trange = ['2023-09-28 06:39:00.000', '2023-09-28 06:39:05.000']  # Adjusted trange to match millisecond precision

# Generate artificial data: summation of three sinusoids with different frequencies
times = pd.date_range(start='2023-09-28 06:39:00', periods=n_points, freq='ms')
# Create a summation of sinusoids for bmag
bmag_extended = (np.sin(2 * np.pi * 1 * np.arange(n_points) / sampling_rate) +  # 1 Hz sine wave
                 0.5 * np.sin(2 * np.pi * 3 * np.arange(n_points) / sampling_rate) +  # 3 Hz sine wave
                 0.25 * np.sin(2 * np.pi * 7 * np.arange(n_points) / sampling_rate))  # 7 Hz sine wave

print(f"Times size: {len(times)}")
print(f"bmag_extended size: {len(bmag_extended)}")
print(f"First few times: {times[:5]}")
print(f"First few bmag_extended values: {bmag_extended[:5]}")

# Clip the data based on trange
times_clipped, bmag = clip_to_original_time_range_multiHeatmap(times, bmag_extended, trange)
print(f"Mask size: {len(times_clipped)}")
print(f"Times clipped size: {len(times_clipped)}")
print(f"Data clipped size: {len(bmag)}")

# Function to generate the heatmap for the entire trange
def generate_heatmap_for_trange(times_clipped, bmag, base_window_seconds, step_size_seconds, num_steps, sampling_rate):
    bool_heatmap = []
    float_heatmap = []
    smoothed_data = None  # Initialize for use in the line plot

    for step in range(num_steps):
        current_window = base_window_seconds + step * step_size_seconds
        smoothed_data = efficient_moving_average_multiHeatmap(
            times_clipped,
            bmag,
            current_window,
            sampling_rate
        )
        
        # Calculate the difference between bmag and smoothed data
        diff_data = bmag - smoothed_data
    
        # Convert to boolean values (1 when bmag < smoothed value, 0 otherwise)
        bool_diff = (diff_data < 0).astype(int)
    
        # Append the results to the heatmap arrays
        bool_heatmap.append(bool_diff)
        float_heatmap.append(diff_data)
    
    # Convert to numpy arrays for plotting
    bool_heatmap = np.array(bool_heatmap)
    float_heatmap = np.array(float_heatmap)
    
    print(f"Boolean heatmap shape: {bool_heatmap.shape}")
    print(f"Float heatmap shape: {float_heatmap.shape}")
    
    # Convert times_clipped to matplotlib date format
    times_num = mdates.date2num(times_clipped)

    # Plotting the line plot and heatmaps
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))  # Add a 3rd plot for raw and smoothed data
    
    # Plot the raw and smoothed data at the top
    plot_raw_and_smoothed_data(times_num, bmag, smoothed_data, axes[0])
    axes[0].set_xlim(times_num[0], times_num[-1])  # Align x-axis with the heatmaps


    # Plot the boolean heatmap
    axes[1].imshow(np.flipud(bool_heatmap), aspect='auto', cmap='binary', 
                   extent=[times_num[0], times_num[-1], base_window_seconds, base_window_seconds + step_size_seconds * num_steps])
    axes[1].set_title('Boolean Heatmap (1 = bmag < smoothed)')
    axes[1].set_ylabel('Smoothing Window (seconds)')
    
    # Add legend/key for the boolean heatmap
    hole_patch = mpatches.Patch(color='black', label='Hole (bmag < smoothed)')
    non_hole_patch = mpatches.Patch(color='white', label='No Hole (bmag >= smoothed)')
    axes[1].legend(handles=[hole_patch, non_hole_patch], loc='upper right')

    # Plot the float heatmap
    im = axes[2].imshow(np.flipud(float_heatmap), aspect='auto', cmap='seismic', 
                        extent=[times_num[0], times_num[-1], base_window_seconds, base_window_seconds + step_size_seconds * num_steps])
    axes[2].set_title('Float Heatmap (bmag - smoothed)')
    axes[2].set_ylabel('Smoothing Window (seconds)')
    axes[2].set_xlabel('Time')

    # Set the date format for the x-axis
    time_format = DateFormatter('%H:%M:%S.%f')
    locator = AutoDateLocator()

    # Apply custom format to each axis
    for ax in axes:
        ax.xaxis.set_major_formatter(plt.FuncFormatter(format_date))
        ax.xaxis.set_major_locator(locator)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Now generate the heatmap with the larger dataset
generate_heatmap_for_trange(times_clipped, bmag, base_window_seconds, step_size_seconds, num_steps, sampling_rate)
